## Modelling

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from python_files.data import GetData
from python_files.basic_preprocessing import BasicPreprocessing
from python_files.preprocessing import Advancedprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import collections
import re
from ast import literal_eval
import itertools
import string
from functools import reduce
from datetime import datetime
import pickle

import xgboost as xgb
import lightgbm as lgb

from scipy import stats
from scipy.stats import pearsonr
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor

from wordcloud import WordCloud

import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

/home/sourav/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = GetData().get_data()['AllMoviesDetailsCleaned']
data.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,production_companies_number,production_countries_number,spoken_languages_number,budget_new,revenue_new,popularity_new,vote_average_new,vote_count_new,belongs_to_collection_x,belongs_to_collection
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,NaN,Villealfa Filmproduction Oy,Finland,...,2,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",NaN,Villealfa Filmproduction Oy,Finland,...,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,NaN,Miramax Films,United States of America,...,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",NaN,Universal Pictures,Japan,...,3,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,NaN,inLoops,Austria,...,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = data.copy()

In [5]:
df_cleaned = BasicPreprocessing.apply(df)
df_cleaned.head()

,id,budget,genres,original_language,original_title,production_companies,production_countries,release_date,revenue,runtime,...,production_companies_number,production_countries_number,spoken_languages_number,budget_new,revenue_new,popularity_new,vote_average_new,vote_count_new,belongs_to_collection_x,collection
2,5,4000000,Crime|Comedy,en,Four Rooms,Miramax Films,United States of America,25/12/1995,4300000,98.0,...,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1
5,9,4000000,Drama,de,Sonntag im August,none,Germany,2/9/2004,4257354,15.0,...,0,1,1,4000000.0,4257354.0,22.475,5.7,2256.0,NaN,1
6,11,11000000,Adventure|Action|Science Fiction,en,Star Wars,Lucasfilm,United States of America,25/05/1977,775398007,121.0,...,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1
7,12,94000000,Animation|Family,en,Finding Nemo,Pixar Animation Studios,United States of America,30/05/2003,940335536,100.0,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1
8,13,55000000,Comedy|Drama|Romance,en,Forrest Gump,Paramount Pictures,United States of America,6/7/1994,677945399,142.0,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
X = df_cleaned.drop(columns = ['revenue'])
y = df_cleaned['revenue']

In [7]:
df_cleaned.columns

Index(['id', 'budget', 'genres', 'original_language', 'original_title',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count',
       'production_companies_number', 'production_countries_number',
       'spoken_languages_number', 'budget_new', 'revenue_new',
       'popularity_new', 'vote_average_new', 'vote_count_new',
       'belongs_to_collection_x', 'collection'],
      dtype='object')

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [37]:
X_train_processed = Advancedprocessing.process(X_train)

/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '13/07/2001' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '27/02/2013' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '22/01/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-re

/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '22/11/2002' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '15/05/2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '16/04/1975' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-re

/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '29/11/2008' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '20/12/1945' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:60: UserWarning: Parsing '25/03/2005' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_copy['release_date'] = pd.to_datetime(df_copy['release_date'], infer_datetime_format=True)
/home/sourav/code/Urus1201/data-movie-re

In [38]:
# Add scoring function
scorer = make_scorer(mean_squared_error)

In [39]:
X_train_processed.drop(columns = ['budget_new', 'revenue_new', 'popularity_new', 'vote_average_new'\
                                 , 'vote_count_new', 'belongs_to_collection_x'], inplace= True)

In [40]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

X_train_processed = X_train_processed.select_dtypes(include=numerics)

In [41]:
# Sustituting the missing runtime value with mean 
from sklearn.impute import SimpleImputer
import numpy as np

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_processed[['runtime']] = imp_mean.fit_transform(X_train_processed[['runtime']])  

In [42]:
# Create validation sets
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train_processed, 
                                                              y_train, 
                                                              test_size = 0.2,
                                                              random_state = 42)

# Print split result
print(X_train_part.shape)
print(X_val.shape)
print(y_train_part.shape)
print(y_val.shape)

(5521, 43)
(1381, 43)
(5521,)
(1381,)


In [43]:
X_train_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5521 entries, 2126 to 860
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   index                        5521 non-null   int64  
 1   budget                       5521 non-null   int64  
 2   runtime                      5521 non-null   float64
 3   vote_average                 5521 non-null   float64
 4   vote_count                   5521 non-null   int64  
 5   production_companies_number  5521 non-null   int64  
 6   production_countries_number  5521 non-null   int64  
 7   spoken_languages_number      5521 non-null   int64  
 8   collection                   5521 non-null   int64  
 9   actor1_gender                5521 non-null   int64  
 10  actor2_gender                5521 non-null   float64
 11  actor3_gender                5521 non-null   int64  
 12  actor4_gender                5521 non-null   int64  
 13  actor5_gender   

### AdaBoost

In [44]:
try:
    # Loading existing pickled file
    best_clf_ada = pickle.load(open('adaboost.sav', 'rb'))
except:
    # Apply supervised machine learning algortihms
    # AdaBoost
    clf_ada = AdaBoostRegressor(random_state=42)

    parameters = {'n_estimators': [10000, 50000, 100000],
                  'learning_rate': [0.001, 0.05, 0.01]}

    grid_init = GridSearchCV(clf_ada, parameters, scoring=scorer)
    grid_fit = grid_init.fit(X_train_part, y_train_part)
    best_clf_ada = grid_fit.best_estimator_
    best_predictions_ada = best_clf_ada.predict(X_val)
    
    # Saving the best AdaBoost model
    pickle.dump(best_clf_ada, open('adaboost.sav', 'wb'))
    
best_predictions_ada = best_clf_ada.predict(X_val)



In [47]:
grid_fit.best_score_

1.709433688629047e+16

### XGBoost

### LightGBM

## Stacking

## Evaluation

### Overfitting

### Feature Importance

## Predictions